In [5]:
import pandas as pd
import numpy as np

# helper functions
def logit(p):
    return np.log(p) - np.log(1 - p)


def invlogit(theta):
    return 1 / (1 + np.exp(-theta))


d = 4


def dirtymultilevel(
    n=np.repeat(100, d), phat=np.repeat(0.5, d), mu0=np.repeat(0, d), V0=None, return_ci=False
):
    if V0 is None:
        # note: V0 should generally be equal to the prior sigma-inverse
        V0 = np.diag(np.repeat(1, len(phat)))
    # phat += 1e-8
    MLE = logit(phat)
    sample_I = np.diag(n * phat * (1 - phat))
    precision_posterior = V0 + sample_I
    Sigma_posterior = np.linalg.inv(precision_posterior)
    mu_posterior = Sigma_posterior @ (sample_I @ MLE + V0 @ mu0)
    # mu_posterior = np.linalg.solve(precision_posterior, sample_I @ MLE + V0 @ mu0)
    ci_dict = {}
    if return_ci:
        # implement 95% CI on each arm
        CI_upper_logit = mu_posterior + 1.96 * np.sqrt(np.diag(Sigma_posterior))
        median_logit = mu_posterior
        CI_lower_logit = mu_posterior - 1.96 * np.sqrt(np.diag(Sigma_posterior))
        conf_logit = pd.DataFrame(
            np.column_stack([CI_lower_logit, median_logit, CI_upper_logit]),
            columns=("lower", "median", "upper"),
        )
        conf_prob = conf_logit.apply(invlogit, axis=1)
        ci_dict['conf_logit']=conf_logit
        ci_dict['conf_prob']=conf_prob
    return dict(
        mu_posterior=mu_posterior,
        Sigma_posterior=Sigma_posterior,
        **ci_dict,
    )


{'mu_posterior': array([-1.04079059,  0.28621066, -0.57397597, -0.57397597]),
 'Sigma_posterior': array([[0.52631579, 0.        , 0.        , 0.        ],
        [0.        , 0.29411765, 0.        , 0.        ],
        [0.        , 0.        , 0.32258065, 0.        ],
        [0.        , 0.        , 0.        , 0.32258065]]),
 'conf_logit':       lower    median     upper
 0 -2.462724 -1.040791  0.381143
 1 -0.776749  0.286211  1.349170
 2 -1.687181 -0.573976  0.539229
 3 -1.687181 -0.573976  0.539229,
 'conf_prob':       lower    median     upper
 0  0.078513  0.260997  0.594149
 1  0.315021  0.571068  0.793994
 2  0.156147  0.360320  0.631633
 3  0.156147  0.360320  0.631633}

In [10]:
n=np.array([10] * 4)
phat = np.array([0.1, 0.6, 0.3, 0.3])
V0 = None
mu0 = np.repeat(0, d)
if V0 is None:
    # note: V0 should generally be equal to the prior sigma-inverse
    V0 = np.diag(np.repeat(1, len(phat)))
# phat += 1e-8
MLE = logit(phat)
sample_I = np.diag(n * phat * (1 - phat))
precision_posterior = V0 + sample_I
precision_posterior

array([[1.9, 0. , 0. , 0. ],
       [0. , 3.4, 0. , 0. ],
       [0. , 0. , 3.1, 0. ],
       [0. , 0. , 0. , 3.1]])

In [ ]:

Sigma_posterior = np.linalg.inv(precision_posterior)
mu_posterior = Sigma_posterior @ (sample_I @ MLE + V0 @ mu0)
# mu_posterior = np.linalg.solve(precision_posterior, sample_I @ MLE + V0 @ mu0)

ci_dict = {}
# implement 95% CI on each arm
CI_upper_logit = mu_posterior + 1.96 * np.sqrt(np.diag(Sigma_posterior))
median_logit = mu_posterior
CI_lower_logit = mu_posterior - 1.96 * np.sqrt(np.diag(Sigma_posterior))
conf_logit = pd.DataFrame(
    np.column_stack([CI_lower_logit, median_logit, CI_upper_logit]),
    columns=("lower", "median", "upper"),
)
conf_prob = conf_logit.apply(invlogit, axis=1)
ci_dict['conf_logit']=conf_logit
ci_dict['conf_prob']=conf_prob
dict(
    mu_posterior=mu_posterior,
    Sigma_posterior=Sigma_posterior,
    **ci_dict,
)